# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [1]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Setting path to train and test data.

In [2]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32
BATCH_SIZE = 128

Loading train data.

In [3]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [4]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [5]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [6]:
train_labels, train_images = load_train_data(TRAIN_DATA_DIR)
val_labels, val_images = load_test_data(TEST_DATA_DIR)

In [19]:
def gen_train_images(directory, files_count):
    datagen = ImageDataGenerator(
        rotation_range=17,
        rescale=1. / 255,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='nearest')
    
    train_generator = datagen.flow_from_directory(directory,
                            shuffle=False,
                            save_to_dir=directory,
                            batch_size=1,
                            save_prefix="keras",
                            save_format="ppm")
    for i in range(files_count):
        train_generator.next()

Creating bar chart for dataset Visualization.

In [ ]:
data = {x:train_labels.count(x) for x in train_labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel('Number of examples')
plt.title('Training dataset')
plt.show()

In [14]:
def conv2d(input_layer, filters, padding):
    return tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=(5, 5),
        padding=padding,
        activation=tf.nn.elu)

def pooling(input_layer):
    return tf.layers.max_pooling2d(
        inputs=input_layer, 
        pool_size=(2, 2), 
        strides=(2, 2),
        padding='same')

with tf.device('/device:GPU:0'):    
    images = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3], name = "images")
    labels = tf.placeholder(tf.int64, [None])
    
    conv1 = conv2d(images, 12, 'valid')
    pool1 = pooling(conv1)
    
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.9)
    conv2 = conv2d(dropout1, 24, 'valid')
    pool2 = pooling(conv2)
    
    dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7)
    #conv3 = conv2d(dropout2, 32, 'same')
    #pool3 = pooling(conv3)
    #dropout3 = tf.layers.dropout(inputs=pool3, rate=0.6)
   
    flat = tf.layers.flatten(dropout2)
    f1 = tf.layers.dense(flat, 300, tf.nn.elu)
    
    #dropout = tf.layers.dropout(inputs=flat, rate=0.5)
    
    logits = tf.layers.dense(f1, len(set(train_labels)), tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1, name="prediction")
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [8]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [9]:
train_labels_arr = np.array(train_labels)
train_images_arr = np.array(train_images)

In [10]:
def validation(train_images, train_labels, val_images, val_labels):
    train_acc = session.run(accuracy,
                    {images: train_images,
                    labels: train_labels})
    val_acc = session.run(accuracy,
                    {images: val_images,
                    labels: val_labels})
    
    return train_acc, val_acc

In [11]:
def training_log(steps, loss, train_acc, val_acc, training_time):
    print('Steps: {}'.format(steps))
    print('Loss: {0} Training accuracy: {1:.2f}% Validation accuracy: {2:.2f}%'
          .format(loss, train_acc * 100, val_acc * 100))
    minutes, seconds = divmod(training_time, 60)
    hours, minutes = divmod(minutes, 60)
    print('Traning time: {0:02.0f}:{1:02.0f}:{2:05.2f}\n'.format(hours, minutes, seconds))  

In [12]:
def train_model(training_images, training_labels, val_images, val_labels, steps):
    t_start = time.time()
    prev_val_acc = 0
    for i in range(steps + 1):
        indexes = np.random.choice(np.arange(len(train_images)), BATCH_SIZE, replace = False)
        batch_images = train_images_arr[indexes]
        batch_labels = train_labels_arr[indexes]
        loss_value = session.run([optimizer, loss],
                                 {images: batch_images,
                                 labels: batch_labels})[1]
        if (i % 50 == 0):
            training_acc, val_acc = validation(batch_images, batch_labels, val_images, val_labels)
            training_log(i, loss_value, training_acc, val_acc, time.time() - t_start)
            
            if (prev_val_acc < val_acc and i % 500 == 0):
                prev_val_acc = val_acc
                saver.save(session, './model/model')
        
                
train_model(train_images, train_labels, val_images, val_labels, 10000)

Steps: 0
Loss: 3.84500265121 Training accuracy: 17.97% Validation accuracy: 8.63%
Traning time: 00:00:47.29

Steps: 50
Loss: 1.92357456684 Training accuracy: 51.56% Validation accuracy: 42.11%
Traning time: 00:00:52.57

Steps: 100
Loss: 1.07906699181 Training accuracy: 74.22% Validation accuracy: 67.61%
Traning time: 00:00:56.23

Steps: 150
Loss: 0.592023909092 Training accuracy: 89.84% Validation accuracy: 76.69%
Traning time: 00:01:00.01

Steps: 200
Loss: 0.590445756912 Training accuracy: 87.50% Validation accuracy: 80.86%
Traning time: 00:01:03.86

Steps: 250
Loss: 0.335897386074 Training accuracy: 89.84% Validation accuracy: 84.46%
Traning time: 00:01:07.67

Steps: 300
Loss: 0.279012739658 Training accuracy: 94.53% Validation accuracy: 85.30%
Traning time: 00:01:11.39

Steps: 350
Loss: 0.184229135513 Training accuracy: 96.09% Validation accuracy: 86.29%
Traning time: 00:01:15.28

Steps: 400
Loss: 0.131153583527 Training accuracy: 98.44% Validation accuracy: 88.26%
Traning time: 00:

Steps: 3600
Loss: 0.0149236917496 Training accuracy: 99.22% Validation accuracy: 91.60%
Traning time: 00:05:24.79

Steps: 3650
Loss: 0.00324146449566 Training accuracy: 100.00% Validation accuracy: 90.74%
Traning time: 00:05:28.92

Steps: 3700
Loss: 0.00743401283398 Training accuracy: 100.00% Validation accuracy: 92.19%
Traning time: 00:05:32.83

Steps: 3750
Loss: 0.00150437140837 Training accuracy: 100.00% Validation accuracy: 91.43%
Traning time: 00:05:36.58

Steps: 3800
Loss: 0.0858402624726 Training accuracy: 100.00% Validation accuracy: 92.27%
Traning time: 00:05:40.63

Steps: 3850
Loss: 0.0124422898516 Training accuracy: 100.00% Validation accuracy: 92.13%
Traning time: 00:05:44.71

Steps: 3900
Loss: 0.00551629625261 Training accuracy: 100.00% Validation accuracy: 91.10%
Traning time: 00:05:48.65

Steps: 3950
Loss: 0.00460925605148 Training accuracy: 100.00% Validation accuracy: 92.13%
Traning time: 00:05:52.45

Steps: 4000
Loss: 0.00148966396227 Training accuracy: 100.00% Valida

Steps: 7100
Loss: 0.00335134798661 Training accuracy: 100.00% Validation accuracy: 92.37%
Traning time: 00:09:56.06

Steps: 7150
Loss: 6.02698528382e-05 Training accuracy: 100.00% Validation accuracy: 93.24%
Traning time: 00:10:00.47

Steps: 7200
Loss: 0.0491222515702 Training accuracy: 99.22% Validation accuracy: 92.53%
Traning time: 00:10:04.32

Steps: 7250
Loss: 0.00124294892885 Training accuracy: 100.00% Validation accuracy: 93.19%
Traning time: 00:10:07.99

Steps: 7300
Loss: 0.00377048947848 Training accuracy: 100.00% Validation accuracy: 91.65%
Traning time: 00:10:11.71

Steps: 7350
Loss: 9.09804512048e-05 Training accuracy: 100.00% Validation accuracy: 92.57%
Traning time: 00:10:15.52

Steps: 7400
Loss: 0.0101155582815 Training accuracy: 100.00% Validation accuracy: 92.16%
Traning time: 00:10:19.98

Steps: 7450
Loss: 0.000607326335739 Training accuracy: 100.00% Validation accuracy: 93.14%
Traning time: 00:10:24.33

Steps: 7500
Loss: 4.785406054e-05 Training accuracy: 100.00% Val

In [15]:
session.close()